#Обучение CycleGAN для переноса стиля в стилистику "Симпсонов"

*Данный ноутбук демонстрирует процесс обучения нейронной сети CycleGAN для задачи переноса стиля. В качестве цели была выбрана стилизация фотографий под рисовку мультсериала «Симпсоны». Работа основана на официальной PyTorch-реализации проекта: [pytorch-CycleGAN-and-pix2pix](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix).*



## 1. Первичная подготовка

### Используемые датасеты

Для обучения модели CycleGAN требуется два независимых набора данных, представляющих два разных домена:

1.  **Домен A (Исходные изображения):** Фотографии реальных людей.
    *   **Название:** `CelebFaces Attributes Dataset (CelebA)`
    *   **Описание:** Крупномасштабный датасет, содержащий более 200 000 фотографий лиц знаменитостей.
    *   **Источник:** [Страница датасета на Kaggle](https://www.kaggle.com/datasets/jessicali9530/celeba-dataset)

2.  **Домен B (Целевой стиль):** Изображения в стилистике «Симпсонов».
    *   **Название:** `The Simpsons Characters Dataset`
    *   **Описание:** Набор данных, включающий более 20 000 изображений персонажей из мультсериала.
    *   **Источник:** [Страница датасета на Kaggle](https://www.kaggle.com/datasets/alexattia/the-simpsons-characters-dataset)

Таким образом, модель будет учиться преобразовывать изображения из домена A (лица) в домен B (Симпсоны) и наоборот.




In [ ]:
# Датасеты д.б. загружены на гуглдиск в каталог my_datasets_zips в виде zip-архивов: celeba.zip и simpsons.zip.

# Для сохранения чекпоинтов обучения, в корне гуглдиска будет создан каталог CycleGAN_Project(позволит возобновить обучение с последнего чекпоинта).

In [2]:
# Подключаем гуглдиск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##2. Подготовка кодовой базы

In [3]:
# Клонируем официальный репозиторий(будем использовать код авторов CycleGAN)
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516 (from 1)
Receiving objects: 100% (2516/2516), 8.20 MiB | 27.00 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.


In [13]:
%cd pytorch-CycleGAN-and-pix2pix/
# /content/pytorch-CycleGAN-and-pix2pix

[Errno 2] No such file or directory: 'pytorch-CycleGAN-and-pix2pix/'
/content/pytorch-CycleGAN-and-pix2pix


In [14]:
# Установка зависимостей
!pip install -r requirements.txt

##3. Подготовка датасета

In [15]:
%cd /content/pytorch-CycleGAN-and-pix2pix/
# print("Current working directory:", os.getcwd())

/content/pytorch-CycleGAN-and-pix2pix


In [16]:
# Настраиваем датасеты
import zipfile
from pathlib import Path

# Датасеты скачаны в папку my_datasets_zips на диске
SIMPSONS_ZIP_PATH = '/content/drive/MyDrive/my_datasets_zips/simpsons.zip'
CELEBA_ZIP_PATH = '/content/drive/MyDrive/my_datasets_zips/celeba.zip'

# Папка для распаковки датасетов(временное хранилище Colab)
RAW_DATA_DIR = Path('/content/datasets_raw')
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

with zipfile.ZipFile(SIMPSONS_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(RAW_DATA_DIR / 'simpsons_raw')
print("Датасет Симпсонов готов.\n")

with zipfile.ZipFile(CELEBA_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(RAW_DATA_DIR / 'celeba_raw')
print("Датасет лиц людей готов.")

Датасет Симпсонов готов.

Датасет лиц людей готов.


In [17]:
# Пути к сырым данным
SOURCE_FACES_DIR = RAW_DATA_DIR/'celeba_raw/img_align_celeba/img_align_celeba'
SOURCE_SIMPSONS_DIR = RAW_DATA_DIR/'simpsons_raw/simpsons_dataset'

In [18]:
DATASET_NAME = 'faces2simpsons'
DATASET_ROOT = Path(f'./datasets/{DATASET_NAME}') # Папка в ./pytorch-CycleGAN-and-pix2pix/datasets/

In [19]:
import os

# Необходимые папки для CycleGAN
os.makedirs(DATASET_ROOT / 'trainA', exist_ok=True)
os.makedirs(DATASET_ROOT / 'trainB', exist_ok=True)
os.makedirs(DATASET_ROOT / 'testA', exist_ok=True)
os.makedirs(DATASET_ROOT / 'testB', exist_ok=True)

In [20]:
import shutil
from sklearn.model_selection import train_test_split

def process_and_split_images(source_dir, train_dir, test_dir, test_size=0.1, num_images=5000):
    all_files = sorted([f for f in source_dir.glob('**/*.jpg')] + [f for f in source_dir.glob('**/*.png')])
    if not all_files:
        print(f"Внимание: в папке {source_dir} не найдено изображений.")
        return

    selected_files = all_files[:min(len(all_files), num_images)]
    train_files, test_files = train_test_split(selected_files, test_size=test_size, random_state=42)

    # ОЧистка
    for f in train_dir.glob('*'):
        f.unlink()
    for f in test_dir.glob('*'):
        f.unlink()

    # Копирование
    for f in train_files:
        shutil.copy(f, train_dir)
    for f in test_files:
        shutil.copy(f, test_dir)

    print(f"Обработана папка {source_dir}: скопировано {len(train_files)} в train, {len(test_files)} в test.")

In [21]:
# Запускаем обработку для домена A (лица людей)
print("\nОбработка домена A (лица людей)...")
process_and_split_images(SOURCE_FACES_DIR, DATASET_ROOT / 'trainA', DATASET_ROOT / 'testA')

# Запускаем обработку для домена B (лица Симпсонов)
print("\nОбработка домена B (лица Симпсонов)...")
process_and_split_images(SOURCE_SIMPSONS_DIR, DATASET_ROOT / 'trainB', DATASET_ROOT / 'testB')

print("\nПодготовка датасета завершена!")


Обработка домена A (лица людей)...
Обработана папка /content/datasets_raw/celeba_raw/img_align_celeba/img_align_celeba: скопировано 4500 в train, 500 в test.

Обработка домена B (лица Симпсонов)...
Обработана папка /content/datasets_raw/simpsons_raw/simpsons_dataset: скопировано 4500 в train, 500 в test.

Подготовка датасета завершена!


In [22]:
!ls -l ./datasets/faces2simpsons/trainA | head -n 5

total 38520
-rw-r--r-- 1 root root 11440 Jul 12 20:12 000001.jpg
-rw-r--r-- 1 root root  7448 Jul 12 20:12 000002.jpg
-rw-r--r-- 1 root root  4253 Jul 12 20:12 000003.jpg
-rw-r--r-- 1 root root 10747 Jul 12 20:12 000004.jpg


##4. Обучение модели

In [14]:
EXPERIMENT_NAME = 'faces2simpsons_cyclegan_v1'

In [15]:
# Путь на Google Диске для сохранения чекпоинтов модели
CHECKPOINTS_DIR = '/content/drive/MyDrive/CycleGAN_Project/checkpoints'

# Количество эпох обучения.
# Эпоха - это один полный проход по всему обучающему датасету.
# Первые N эпох обучение идет с постоянной скоростью обучения (learning rate).
N_EPOCHS = 50

# Количество эпох, после которых скорость обучения начнет линейно снижаться до нуля.
# Общее количество эпох будет N_EPOCHS + N_EPOCHS_DECAY.
N_EPOCHS_DECAY = 50 # Итого 50 + 50 = 100 эпох

# Как часто сохранять чекпоинт модели на Google Диск.
# `5` означает, что модель будет сохраняться в конце 5-й, 10-й, 15-й... эпохи.
SAVE_EPOCH_FREQ = 1

In [ ]:
# # Запуск обучения
# !python train.py \
#   --dataroot ./datasets/faces2simpsons \
#   --name {EXPERIMENT_NAME} \
#   --model cycle_gan \
#   --checkpoints_dir {CHECKPOINTS_DIR} \
#   --gpu_ids 0 \
#   --n_epochs {N_EPOCHS} \
#   --n_epochs_decay {N_EPOCHS_DECAY} \
#   --save_epoch_freq {SAVE_EPOCH_FREQ} \
#   --display_id -1 \
#   # --continue_train \
#   --epoch latest

In [ ]:
# Команда для ПРОДОЛЖЕНИЯ обучения
!python train.py \
  --dataroot ./datasets/faces2simpsons \
  --name {EXPERIMENT_NAME} \
  --model cycle_gan \
  --checkpoints_dir {CHECKPOINTS_DIR} \
  --gpu_ids 0 \
  --n_epochs {N_EPOCHS} \
  --n_epochs_decay {N_EPOCHS_DECAY} \
  --save_epoch_freq {SAVE_EPOCH_FREQ} \
  --display_id -1 \
  --continue_train \
  --epoch latest

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: /content/drive/MyDrive/CycleGAN_Project/checkpoints	[default: ./checkpoints]
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: ./datasets/faces2simpsons     	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            	[default: 1]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                        

##5. Тестирование и просмотр результатов

In [23]:
# !pwd # /content/pytorch-CycleGAN-and-pix2pix
# ls -la ./datasets/faces2simpsons
# CHECKPOINTS_DIR = '/content/drive/MyDrive/CycleGAN_Project/checkpoints'
# !ls -la drive/MyDrive/CycleGAN_Project/checkpoints
!ls -la /content/drive/MyDrive/CycleGAN_Project/checkpoints/faces2simpsons_cyclegan_v1

total 1216838
-rw------- 1 root root 11062946 Jul  9 19:57 10_net_D_A.pth
-rw------- 1 root root 11062946 Jul  9 19:57 10_net_D_B.pth
-rw------- 1 root root 45531874 Jul  9 19:57 10_net_G_A.pth
-rw------- 1 root root 45531874 Jul  9 19:57 10_net_G_B.pth
-rw------- 1 root root 11062932 Jul 12 17:32 1_net_D_A.pth
-rw------- 1 root root 11062932 Jul 12 17:32 1_net_D_B.pth
-rw------- 1 root root 45531822 Jul 12 17:32 1_net_G_A.pth
-rw------- 1 root root 45531822 Jul 12 17:32 1_net_G_B.pth
-rw------- 1 root root 11062932 Jul 12 18:10 2_net_D_A.pth
-rw------- 1 root root 11062932 Jul 12 18:10 2_net_D_B.pth
-rw------- 1 root root 45531822 Jul 12 18:10 2_net_G_A.pth
-rw------- 1 root root 45531822 Jul 12 18:10 2_net_G_B.pth
-rw------- 1 root root 11062932 Jul 12 18:48 3_net_D_A.pth
-rw------- 1 root root 11062932 Jul 12 18:48 3_net_D_B.pth
-rw------- 1 root root 45531822 Jul 12 18:48 3_net_G_A.pth
-rw------- 1 root root 45531822 Jul 12 18:48 3_net_G_B.pth
-rw------- 1 root root 11062932 Jul 12

In [24]:
# Тестирование
# EXPERIMENT_NAME = 'faces2simpsons_cyclegan_v1'
# # !python test.py \
# #   --dataroot ./datasets/faces2simpsons/testA \
# #   --name {EXPERIMENT_NAME} \
# #   --model cycle_gan \
# #   --checkpoints_dir {CHECKPOINTS_DIR} \
# #   --no_dropout \
# #   --results_dir ./results/{EXPERIMENT_NAME}  \
# #   --gpu_ids -1
# !python test.py --dataroot ./datasets/faces2simpsons/testA --name {EXPERIMENT_NAME} --model cycle_gan --checkpoints_dir {CHECKPOINTS_DIR} --no_dropout --results_dir ./results/{EXPERIMENT_NAME} --gpu_ids -1

EXPERIMENT_NAME = 'faces2simpsons_cyclegan_v1'
CHECKPOINTS_DIR = '/content/drive/MyDrive/CycleGAN_Project/checkpoints'
# Указываем путь к faces2simpsons, а не к testA внутри нее.
!python test.py --dataroot ./datasets/faces2simpsons --name {EXPERIMENT_NAME} --model cycle_gan --checkpoints_dir {CHECKPOINTS_DIR} --no_dropout --results_dir ./results/{EXPERIMENT_NAME} --gpu_ids -1

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: /content/drive/MyDrive/CycleGAN_Project/checkpoints	[default: ./checkpoints]
                crop_size: 256                           
                 dataroot: ./datasets/faces2simpsons     	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: -1                            	[default: 0]
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: Non

In [ ]:
# Домен A — это реальные лица.
# Домен B — это Симпсоны.
# Нам нужна работа генератора, который переводит A -> B.

In [25]:
# Отображение нескольких результатов
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path

# Определяем имя эксперимента, если еще не определено
EXPERIMENT_NAME = 'faces2simpsons_cyclegan_v1'

# Правильный путь к результатам
results_path = Path(f'./results/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}/test_latest/images')

print(f"Поиск изображений в папке: {results_path}")

# --- ИЗМЕНЕНИЕ 1: Ищем файлы, заканчивающиеся на _fake_B.png ---
fake_images = sorted(list(results_path.glob('*_fake_B.png')))[0:5]

if not fake_images:
    print("\nНе найдено сгенерированных изображений! Проверьте, что имена файлов верны.")
else:
    print(f"\nНайдено {len(fake_images)} изображений. Отображение...")
    for img_path in fake_images:

        real_img_path = Path(str(img_path).replace('_fake_B', '_real_A'))

        if not real_img_path.exists():
            print(f"Не найден оригинал ({real_img_path}) для {img_path}")
            continue

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

        ax1.imshow(mpimg.imread(real_img_path))
        ax1.set_title('Original Face (Real A)')
        ax1.axis('off')

        ax2.imshow(mpimg.imread(img_path))
        ax2.set_title('Stylized (Fake B)')
        ax2.axis('off')

        plt.show()

Output hidden; open in https://colab.research.google.com to view.